<a href="https://colab.research.google.com/github/mpurkeypile-usd/AAI590-Final_Project/blob/main/AAI590_Final_Project_MPurkeypile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Matt Purkeypile  
AAI590 Capstone: Final Project  
August 2025  

# Breast Tumor Classification Using Quantum Neural Networks

## Setup and library loading

In [ ]:
# based on the prototype quantum neural network I did as a proof of concept
# back in April to get the ok for this approach. Notebook for that:
# https://colab.research.google.com/drive/1c2f93b4DxPKU9s4CI8PRZdV_AEoq8ekn
# necessary installs
# cirq works on Python 3.10 and later:
# https://github.com/quantumlib/Cirq/blob/main/docs/start/install.md

#############################################
# Do NOT restart the runtime when prompted! #
#############################################

# based on: https://www.tensorflow.org/quantum/install
# then corrected with AI recommendations
!pip3 install --upgrade pip
!pip3 install tensorflow==2.15.0
!pip3 install -U tensorflow-quantum
!pip install cirq==1.1.0 cirq-google==1.1.0
!pip3 install sympy
!pip install numpy==1.23.5 # Install numpy==1.23.5 to resolve incompatibility

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq    # reference: https://www.tensorflow.org/quantum
import cirq                         # reference: https://quantumai.google/cirq/
import cirq_google
from cirq.contrib.svg import SVGCircuit
import sympy
import numpy as np
import pandas as pd
from google.colab import drive
import collections
import matplotlib.pyplot as plt

## Data Cleaning

In [ ]:
# first load the data
# note that the file is also in the github repo and could
# be moved into this location or pulled from there.
drive.mount('/content/gdrive')
DATASET_PATH = '/content/gdrive/My Drive/Colab Notebooks/breast-cancer.csv'

raw_data = pd.read_csv(DATASET_PATH)
raw_data.head()

In [ ]:
# any missing values?
raw_data.isna().sum()

## Exploratory Data Analysis

## Model Design and Building

## Model Training

## Model Optimization

## Model Analysis